In [1]:
%cd /home/q123/Desktop/explo

### local imports 
from src.environment import EnvironmentObjective
from src.vanillagp import step
from src.policy import MLP

### botorch
from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

### general imports
import torch
import gym


/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### initialize policy
mlp = MLP([3,1])

# Initialize environment

objective_env = EnvironmentObjective(
  env=gym.make("Pendulum-v1"),
  policy=mlp,
  manipulate_state=None,
  manipulate_reward=None,
)

### initialize train_x, train_y
train_x = torch.rand(1,mlp.len_params) ## [n_trials,n_params]
train_y = [objective_env.run(p) for p in train_x]
train_y = torch.Tensor(train_y).reshape(-1,1)  ## [n_trials,1]

### initialize model
model = SingleTaskGP(train_X=train_x, train_Y=train_y)

### now we loop :
max_iter = 1000

for i in range(max_iter):

  step(model,objective_env)

  if i % 100 == 0:

    best_val = model.train_targets.max()
    curr_val = model.train_targets[-1]
    print(f'curr {curr_val} max {best_val}')

curr -1605.945556640625 max -1417.66259765625
curr -1655.7308349609375 max -633.647216796875
curr -1420.776123046875 max -633.647216796875


KeyboardInterrupt: 

In [ ]:
model.covar_module["_modules"].base_kernel.__dict__

{'training': True,
 '_parameters': OrderedDict([('raw_lengthscale',
               Parameter containing:
               tensor([[ 1.5522,  0.5471, -1.8617]], requires_grad=True))]),
 '_buffers': OrderedDict([('active_dims', None)]),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict([(645,
               <bound method Module._load_state_hook_ignore_shapes of MaternKernel(
                 (lengthscale_prior): GammaPrior()
                 (raw_lengthscale_constraint): Positive()
                 (distance_module): Distance()
               )>)]),
 '_modules': OrderedDict([('lengthscale_prior', GammaPrior()),
              ('raw_lengthscale_constraint', Positive()),
              ('distance_module', Distance())]),
 '_added_loss_terms': OrderedDict(),
 '_priors': OrderedDict([('lengt